In [105]:
freq = int((1 / 10) * 1e9)

In [106]:
freq

100000000

In [43]:
import rosbag
import rospy
import json
import struct

In [44]:
bag_file = rosbag.Bag('/home/devel/Data/lidar_cam_calib_db_project.bag')
msg_count = bag_file.get_message_count()

In [45]:
DATA_TYPES = {
    1: 'b', # INT8
    2: 'B', # UINT8
    3: 'h', # INT16
    4: 'H', # UINT16
    5: 'i', # INT32
    6: 'I', # UINT32
    7: 'f', # FLOAT32
    8: 'd'  # FLOAT64
}
def _get_format_str(format: int) -> str:

    if format not in DATA_TYPES.keys():
        raise ValueError(f"Error processing Pointcloud2: Unknown format: {format}")

    endian_char =  '<'
    type_char = DATA_TYPES[format]

    return endian_char + type_char

In [46]:
_get_format_str(int(7))

'<f'

In [56]:
i = 0
i:(i + 32-1)
i

0

In [58]:
import os
radar1_seq = 0
for topic, msg, t in bag_file.read_messages():
    
    # Sensor is a radar
    if topic == "/ti_mmwave/radar_scan_pcl_1":
        points = []
        fn_seq = f'seq_{radar1_seq}.json'
        path = os.path.join('../data/radar1', fn_seq)
        i = 0
        for _ in range(msg.width * msg.height):
            point_data = msg.data[i:(i + msg.point_step - 1)]
            i += msg.point_step
            
            point = dict()

            for field in msg.fields:
                format_str = '<f'
                ba = bytearray(point_data[field.offset:field.offset + 4])
                data = struct.unpack(format_str, ba)
                point[field.name] = data
            points.append(point)
            with open (path, 'w') as f:
                json.dump(points, f, indent=4, sort_keys=True)
        radar1_seq += 1
            
            

In [41]:
points

[{'x': (0.985301673412323,),
  'y': (0.6228880882263184,),
  'z': (0.12623858451843262,),
  'intensity': (14.699999809265137,),
  'velocity': (0.23872025310993195,),
  'range': (1.1724953651428223,)},
 {'x': (0.9933264255523682,),
  'y': (0.6228880882263184,),
  'z': (0.007624021265655756,),
  'intensity': (18.299999237060547,),
  'velocity': (-0.23872025310993195,),
  'range': (1.1724952459335327,)}]

In [109]:
from dataclasses import dataclass

In [110]:
@dataclass
class LastFrame:
    """
    Object for storing received data.
    """
    frame_counter: 0
    timestamp: None
    data: dict()

    def has_enough_data(self, topics: dict) -> bool:
        """
        Checks if we have collected data from all topics that
        we are monitoring.

        Args:
            topics (dict): Dictionary object containing valid topics

        Returns:
            bool: True if enough data has been collected. False otherwise
        """
        if len(self.data.keys()) != len(topics):
            return False

        return True

In [111]:
last_frame_msg = LastFrame(0, 0, dict())

In [112]:
last_frame_msg

LastFrame(frame_counter=0, timestamp=0, data={})

In [113]:
for topic, msg, t in bag_file.read_messages():
    if topic == '/ti_mmwave/radar_scan_pcl_1':
        last_frame_msg.data[topic] = msg
    if topic == '/image_color_rect/compressed':
        last_frame_msg.data[topic] = msg.data
    time_ns = rospy.rostime.Time.to_nsec(t)
    

In [114]:
image_data = last_frame_msg.data['/image_color_rect/compressed']

In [115]:
type(image_data)

bytes

In [116]:
from PIL import Image
import io
import cv2
import numpy as np

In [117]:
img_array = cv2.imdecode(np.frombuffer(image_data, np.uint8), -1)
rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)


In [118]:
rgb

array([[[131, 100, 132],
        [130,  99, 131],
        [132, 102, 136],
        ...,
        [171, 133, 132],
        [155, 117, 116],
        [158, 120, 119]],

       [[134, 103, 135],
        [130, 101, 132],
        [129,  99, 133],
        ...,
        [171, 133, 132],
        [157, 119, 118],
        [161, 123, 122]],

       [[133, 104, 134],
        [130, 101, 131],
        [127, 100, 131],
        ...,
        [164, 125, 126],
        [153, 114, 115],
        [155, 116, 117]],

       ...,

       [[209, 175, 202],
        [206, 172, 199],
        [204, 170, 195],
        ...,
        [203, 164, 195],
        [206, 167, 198],
        [208, 169, 200]],

       [[206, 172, 199],
        [204, 170, 197],
        [202, 168, 193],
        ...,
        [204, 164, 198],
        [205, 165, 199],
        [205, 165, 199]],

       [[206, 172, 199],
        [204, 170, 197],
        [204, 170, 195],
        ...,
        [205, 165, 200],
        [204, 164, 199],
        [203, 163, 198]]

In [119]:
import velodyne_decoder as vd

config = vd.Config(model='VLP-32C')
lidar_topics = ['/velodyne_packets']
cloud_arrays = []
for stamp, points, topic in vd.read_bag(bag_file, config, lidar_topics):
    cloud_arrays.append(points)

In [120]:
len(cloud_arrays)

284

In [121]:
from velodyne_decoder_pylib import *
decoder = ScanDecoder(config)
for topic, msg, t in bag_file.read_messages():
    if topic == '/velodyne_packets':
        for packet in msg.packets:
            packet.data = bytearray(packet.data)
        points = decoder.decode_message(msg, True)
        last_frame_msg.data["/velodyne_packets"] = points

In [122]:
for sensor in last_frame_msg.data:
    print(sensor)

/image_color_rect/compressed
/ti_mmwave/radar_scan_pcl_1
/velodyne_packets


In [123]:
f"f_{last_frame_msg.frame_counter}" + '_rgb'

'f_0_rgb'

In [124]:
last_frame_msg.data["/velodyne_packets"]

array([(0.03676616, -1.2610446, -0.5882847 ,  2.,  0, -0.09949732),
       (0.13766032, -1.079089 , -0.01898822,  4., 17, -0.09949732),
       (0.05756208, -1.9743248, -0.05748289, 59., 15, -0.09949502), ...,
       (0.20471144, -2.9951098,  0.8044096 , 87., 31,  0.00064051),
       (0.05785955, -2.9862015,  0.5445639 ,  6., 30,  0.00064282),
       (0.08425938, -1.2327895, -0.02875324,  6., 16,  0.00064282)],
      dtype={'names': ['x', 'y', 'z', 'intensity', 'ring', 'time'], 'formats': ['<f4', '<f4', '<f4', '<f4', '<u2', '<f4'], 'offsets': [0, 4, 8, 16, 20, 24], 'itemsize': 32})

In [104]:
time_ns

1670873707770620163

In [105]:
last_frame_msg.timestamp

0

In [106]:
last_frame_msg.frame_counter

0

In [107]:
import itertools

In [108]:
sensor = last_frame_msg.data

In [16]:
for sensor in last_frame_msg.data:
    print(sensor[0])
    sensor_name = sensor[1:]

/


In [41]:
last_frame_msg.data["/velodyne_packets"]

array([[ 3.6766160e-02, -1.2610446e+00, -5.8828467e-01,  2.0000000e+00,
         0.0000000e+00, -9.9497318e-02],
       [ 1.3766032e-01, -1.0790890e+00, -1.8988218e-02,  4.0000000e+00,
         1.7000000e+01, -9.9497318e-02],
       [ 5.7562076e-02, -1.9743248e+00, -5.7482895e-02,  5.9000000e+01,
         1.5000000e+01, -9.9495016e-02],
       ...,
       [ 2.0471144e-01, -2.9951098e+00,  8.0440962e-01,  8.7000000e+01,
         3.1000000e+01,  6.4051198e-04],
       [ 5.7859547e-02, -2.9862015e+00,  5.4456389e-01,  6.0000000e+00,
         3.0000000e+01,  6.4281601e-04],
       [ 8.4259383e-02, -1.2327895e+00, -2.8753242e-02,  6.0000000e+00,
         1.6000000e+01,  6.4281601e-04]], dtype=float32)

In [44]:
with open('test.bin', 'wb') as f:
    last_frame_msg.data["/velodyne_packets"].tofile(f)
    

In [46]:
import pickle
with open('test.pkl', 'wb') as f:
    last_frame_msg.data["/velodyne_packets"].tofile(f)

In [20]:
valid_topics = {'lidar': ['/velodyne_points'], 'camera': ['/pycam/camera/image_raw/compressed'], 'radar': ['/ti_mmwave/radar_scan_pcl_1']}


In [38]:
valid_topics.get('camera')[0]

'/pycam/camera/image_raw/compressed'

In [27]:
from message_filters import Subscriber, TimeSynchronizer, ApproximateTimeSynchronizer
from sensor_msgs.msg import Image
from sensor_msgs.msg import PointCloud2

In [45]:
raspi_cam_sub = Subscriber(valid_topics.get('camera')[0], Image)
radar_1_sub = Subscriber(valid_topics.get('radar')[0], PointCloud2)
lidar_sub = Subscriber(valid_topics.get('lidar')[0], PointCloud2)

In [46]:
sync = ApproximateTimeSynchronizer([raspi_cam_sub,
                                            radar_1_sub,
                                            lidar_sub], queue_size=10, slop=0.1)

In [48]:
sync.registerCallback(call_back)
def call_back(raspi, radar, lidar):
    raspi = raspi
    print(raspi)
    return

In [192]:
import pickle
with open('/home/devel/finest_data/lidar_raspi_radar_all_0/f_11_velodyne_packets.pkl', 'rb') as _f:
    data = pickle.load(_f)

In [195]:
data[:,:3]

array([[-0.35854566,  1.550562  , -0.7421177 ],
       [-1.2999518 ,  3.8582716 , -0.07106619],
       [-0.85664517,  3.7046366 , -0.11066039],
       ...,
       [-0.90039855,  3.266649  ,  0.9079372 ],
       [-0.7980963 ,  3.5675523 ,  0.66653466],
       [-1.0445457 ,  3.7896152 , -0.09147067]], dtype=float32)

In [125]:
import numpy as np

In [ ]:
pts = np.fromfile('/home/devel/D')

In [81]:
all = list()
a = {
        'topic': 'a',
        'file': 'b',
        'data': 'c'
    }
b = {
        'topic': 'd',
        'file': 'e',
        'data': 'f'
    }


In [101]:
all = [a, b]
all

[{'topic': 'a', 'file': 'b', 'data': 'c'},
 {'topic': 'd', 'file': 'e', 'data': 'f'}]

In [82]:
all = dict(list(a.items())+list(b.items()))

{'topic': 'd', 'file': 'e', 'data': 'f'}

In [53]:
frame_info = list()
for i in range(0,2):
    a = {
            'topic': i,
            'file': i,
            'data': i
        }
    frame_info.append(a)

[{'topic': 0, 'file': 0, 'data': 0}, {'topic': 1, 'file': 1, 'data': 1}]

In [56]:
d1={'a':[1,2,3], 'b':[4,5,6], 'c':[7,8,9]}
d2 = {'a':[10,11,12], 'b':[13,14,15], 'c':[16,17,18]}
d3 = {}
for k in d1.keys():
    d3.update( {k : []} )
    for i in d1[k]:
        d3[k].append(i)
    for j in d2[k]:
        d3[k].append(j)
print(d3)

{'a': [1, 2, 3, 10, 11, 12], 'b': [4, 5, 6, 13, 14, 15], 'c': [7, 8, 9, 16, 17, 18]}


In [5]:
import pandas as pd
import os
import cv2
import sys
import pickle
sys.path.append('/home/devel/workspaces/finest_mobility/src/data_unpack/src')
import lidar_camera_projection as lcp

In [6]:
base_dir = '/home/devel/finest_data/lidar_cam_calib_db_project'
proj_img_dir = os.path.join(base_dir, 'proj_img')
proj_pkl_dir = os.path.join(base_dir, 'proj_pkl')

In [7]:
for d in [proj_img_dir, proj_pkl_dir]:
    if not os.path.exists(d):
        os.mkdir(d)

In [8]:
df_lidar_to_cam = pd.read_csv('/home/devel/finest_data/lidar_cam_calib_db_project/lidar-to-cam-seq-sync.csv')

In [9]:
df_lidar_to_cam

,lidar,cam
0,0,0
1,1,1
2,2,3
3,3,4
4,4,5
...,...,...
1439,1439,1902
1440,1440,1904
1441,1441,1905
1442,1442,1906


In [26]:
df_lidar_to_cam.set_index('lidar').loc[[4]].values.reshape(-1)

array([5])

In [29]:
df_lidar_to_cam.lidar.values[-1]

1443

In [11]:
lidar_seq = df_lidar_to_cam.lidar.values
lidar_seq

array([   0,    1,    2, ..., 1441, 1442, 1443])

In [16]:
for seq in lidar_seq:
    cam_seq = df_lidar_to_cam.set_index('lidar').loc[[seq]].values.reshape(-1)
    for sq in cam_seq:
        img = cv2.imread(f'/home/devel/finest_data/lidar_cam_calib_db_project/camera/seq_{sq}_rgb.png',
                         cv2.IMREAD_UNCHANGED )
    with open(f'/home/devel/finest_data/lidar_cam_calib_db_project/lidar/seq_{seq}_lidar.pkl', 'rb') as f:
        lidar_points = pickle.load(f)
    img_out = os.path.join(proj_img_dir, f'./lidar_{seq}_to_rgb_{cam_seq}.png')
    pkl_out = os.path.join(proj_pkl_dir, f'./lidar_{seq}_to_rgb_{cam_seq}.pkl')
    lcp.lidar_projection(img, lidar_points, pkl_out, img_out)
    print(f'Done seq {seq}')

Done seq 0
Done seq 1
Done seq 2
Done seq 3
Done seq 4
Done seq 5
Done seq 6
Done seq 7
Done seq 8
Done seq 9
Done seq 10
Done seq 11
Done seq 12
Done seq 13
Done seq 14
Done seq 15
Done seq 16
Done seq 17
Done seq 18
Done seq 19
Done seq 20
Done seq 21
Done seq 22
Done seq 23
Done seq 24
Done seq 25
Done seq 26
Done seq 27
Done seq 28
Done seq 29
Done seq 30
Done seq 31
Done seq 32
Done seq 33
Done seq 34
Done seq 35
Done seq 36
Done seq 37
Done seq 38
Done seq 39
Done seq 40
Done seq 41
Done seq 42
Done seq 43
Done seq 44
Done seq 45
Done seq 46
Done seq 47
Done seq 48
Done seq 49
Done seq 50
Done seq 51
Done seq 52
Done seq 53
Done seq 54
Done seq 55
Done seq 56
Done seq 57
Done seq 58
Done seq 59
Done seq 60
Done seq 61
Done seq 62
Done seq 63
Done seq 64
Done seq 65
Done seq 66
Done seq 67
Done seq 68
Done seq 69
Done seq 70
Done seq 71
Done seq 72
Done seq 73
Done seq 74
Done seq 75
Done seq 76
Done seq 77
Done seq 78
Done seq 79
Done seq 80
Done seq 81
Done seq 82
Done seq 83
Do

Done seq 639
Done seq 640
Done seq 641
Done seq 642
Done seq 643
Done seq 644
Done seq 645
Done seq 646
Done seq 647
Done seq 648
Done seq 649
Done seq 650
Done seq 651
Done seq 652
Done seq 653
Done seq 654
Done seq 655
Done seq 656
Done seq 657
Done seq 658
Done seq 659
Done seq 660
Done seq 661
Done seq 662
Done seq 663
Done seq 664
Done seq 665
Done seq 666
Done seq 667
Done seq 668
Done seq 669
Done seq 670
Done seq 671
Done seq 672
Done seq 673
Done seq 674
Done seq 675
Done seq 676
Done seq 677
Done seq 678
Done seq 679
Done seq 680
Done seq 681
Done seq 682
Done seq 683
Done seq 684
Done seq 685
Done seq 686
Done seq 687
Done seq 688
Done seq 689
Done seq 690
Done seq 691
Done seq 692
Done seq 693
Done seq 694
Done seq 695
Done seq 696
Done seq 697
Done seq 698
Done seq 699
Done seq 700
Done seq 701
Done seq 702
Done seq 703
Done seq 704
Done seq 705
Done seq 706
Done seq 707
Done seq 708
Done seq 709
Done seq 710
Done seq 711
Done seq 712
Done seq 713
Done seq 714
Done seq 715

Done seq 1250
Done seq 1251
Done seq 1252
Done seq 1253
Done seq 1254
Done seq 1255
Done seq 1256
Done seq 1257
Done seq 1258
Done seq 1259
Done seq 1260
Done seq 1261
Done seq 1262
Done seq 1263
Done seq 1264
Done seq 1265
Done seq 1266
Done seq 1267
Done seq 1268
Done seq 1269
Done seq 1270
Done seq 1271
Done seq 1272
Done seq 1273
Done seq 1274
Done seq 1275
Done seq 1276
Done seq 1277
Done seq 1278
Done seq 1279
Done seq 1280
Done seq 1281
Done seq 1282
Done seq 1283
Done seq 1284
Done seq 1285
Done seq 1286
Done seq 1287
Done seq 1288
Done seq 1289
Done seq 1290
Done seq 1291
Done seq 1292
Done seq 1293
Done seq 1294
Done seq 1295
Done seq 1296
Done seq 1297
Done seq 1298
Done seq 1299
Done seq 1300
Done seq 1301
Done seq 1302
Done seq 1303
Done seq 1304
Done seq 1305
Done seq 1306
Done seq 1307
Done seq 1308
Done seq 1309
Done seq 1310
Done seq 1311
Done seq 1312
Done seq 1313
Done seq 1314
Done seq 1315
Done seq 1316
Done seq 1317
Done seq 1318
Done seq 1319
Done seq 1320
Done s

In [ ]:

for seq in range(373):
        img0 = cv2.imread(f'/home/devel/workspaces/finest_mobility/src/data_unpack/data/camera/seq_{seq}_rgb.png',
                  cv2.IMREAD_UNCHANGED)
        with open(f'/home/devel/workspaces/finest_mobility/src/data_unpack/data/lidar/seq_{seq}_lidar.pkl','rb') as _fd:
            points0 = pickle.load(_fd)
        img_out = os.path.join(proj_img_dir, f'./img_out_seq_{seq}.png')
        pkl_out = os.path.join(proj_pkl_dir, f'./pkl_out_seq_{seq}.pkl')
        lcp.lidar_projection(img0, points0, pkl_out, img_out)
        print(f'Done seq {seq}')